### Обучение пайплайна

1. Загрузим данные 
2. Соберем пайплайн с препроцессингом на pricing данных
3. Обучим model и сохраним на диск предобученный пайплайн

In [ ]:
!pip install catboost

In [3]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler 
from sklearn.impute import SimpleImputer
from sklearn.utils import class_weight
import sklearn.datasets
from sklearn.metrics import precision_score,recall_score
from datetime import datetime
from catboost import CatBoostClassifier

In [19]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [20]:
DATA_PATH = '/gdrive/My Drive/Ml_In_Business/kursovaya/'
training_dir = f"{DATA_PATH}/catboost_info/" 
DATA_PATH

'/gdrive/My Drive/Ml_In_Business/kursovaya/'

In [21]:
TARGET = 'TARGET'
BUY_SFX = '_BUY'
SELL_SFX = '_SELL'

TARGET_POSITIVE = f'TARGET{BUY_SFX}'
TARGET_NEGATIVE = f'TARGET{SELL_SFX}'
TARGET, TARGET_POSITIVE, TARGET_NEGATIVE

('TARGET', 'TARGET_BUY', 'TARGET_SELL')

In [22]:
def datetime_to_title():
    now = datetime.now()
    return now.strftime("%d_%m_%Y_%H%M%S")

Загрузим данные

In [23]:
# https://drive.google.com/file/d/1qlCS0RF5mWcW3dKwRX9Eh1yco7lK6zd7/view?usp=sharing
df_full = pd.read_csv(f"{DATA_PATH}GOLD_H1.csv")
df_full.head(3)

,datetime,open,high,low,close,tick_volume,spread,TARGET,t_price,fractals_high,fractals_low,RSI_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,MOM_14,CCI_14_0.015,EMA_4_ema4o,EMA_5_ema5c
0,2018-08-14 21:00:00,1193.78,1194.62,1193.47,1194.21,1420,160,-1.0,1194.62,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-08-14 22:00:00,1194.19,1194.66,1193.74,1194.25,966,160,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-08-14 23:00:00,1194.25,1194.56,1193.73,1193.87,568,180,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_full[TARGET].fillna(value=0, inplace=True)
df_full['t_price'].fillna(value=0, inplace=True)
df_full[TARGET].value_counts()

 0.0    11927
-1.0     4104
 1.0     3969
Name: TARGET, dtype: int64

Transform 3-class target to binary

In [25]:
df_full = df_full.dropna(axis=0)

In [26]:
df_full.shape

(19969, 18)

In [27]:
df_full.describe()

,open,high,low,close,tick_volume,spread,TARGET,t_price,RSI_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,MOM_14,CCI_14_0.015,EMA_4_ema4o,EMA_5_ema5c
count,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,1.996900e+04,1.996900e+04,19969.000000,19969.000000,19969.000000,19969.000000
mean,1607.115533,1609.178871,1604.962166,1607.148834,4558.063248,151.670339,-0.006660,649.308401,51.470614,5.070994e+01,5.070844e+01,0.443061,7.073828,1607.067363,1607.084649
std,241.933072,242.508071,241.265748,241.924089,3162.086840,18.950143,0.635415,804.810194,12.583301,3.228863e+01,3.094511e+01,11.954756,112.189621,241.935547,241.928386
min,1172.260000,1172.930000,1170.690000,1172.680000,3.000000,0.000000,-1.000000,0.000000,9.572869,-2.107943e-13,-1.892807e-13,-113.700000,-435.143714,1171.331451,1171.992824
25%,1389.428000,1391.790000,1387.400000,1389.470000,2155.000000,140.000000,0.000000,0.000000,43.106453,2.066897e+01,2.221752e+01,-4.710000,-75.883298,1389.780168,1389.948861
50%,1697.530000,1700.500000,1694.720000,1697.590000,3750.000000,160.000000,0.000000,0.000000,51.312263,5.060952e+01,5.051812e+01,0.630000,10.762012,1697.666317,1697.866754
75%,1806.620000,1808.510000,1804.660000,1806.640000,6118.000000,160.000000,0.000000,1511.090000,59.966833,8.077973e+01,7.963499e+01,6.070000,91.158711,1806.349608,1806.319019
max,2070.320000,2074.920000,2068.290000,2070.230000,20898.000000,1397.000000,1.000000,2074.920000,91.148616,1.000000e+02,1.000000e+02,84.020000,436.741254,2067.513122,2067.166346


In [28]:
df_full[TARGET_POSITIVE] = df_full[TARGET]
#  set 0 where class is -1
df_full.loc[df_full[TARGET_POSITIVE] ==-1,  TARGET_POSITIVE] = 0

df_full[TARGET_NEGATIVE] = df_full[TARGET]
#  set 0 where class is 1
df_full.loc[df_full[TARGET_NEGATIVE] == 1,  TARGET_NEGATIVE] = 0

In [29]:
df_full.head(3)

,datetime,open,high,low,close,tick_volume,spread,TARGET,t_price,fractals_high,fractals_low,RSI_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,MOM_14,CCI_14_0.015,EMA_4_ema4o,EMA_5_ema5c,TARGET_BUY,TARGET_SELL
31,2018-08-16 06:00:00,1172.26,1172.93,1170.69,1172.68,3400,160,0.0,0.0,False,False,34.016815,100.0,72.430645,-12.50,-60.901468,1171.331451,1171.992824,0.0,0.0
32,2018-08-16 07:00:00,1172.67,1174.32,1171.51,1173.93,2317,160,0.0,0.0,False,False,37.290539,100.0,88.931516,-7.20,-25.768602,1171.866871,1172.638549,0.0,0.0
33,2018-08-16 08:00:00,1173.93,1178.60,1173.67,1177.65,4443,160,-1.0,1178.6,True,False,45.893977,100.0,100.000000,-1.33,63.547341,1172.692122,1174.309033,0.0,-1.0


In [30]:
df_full[TARGET_POSITIVE].value_counts()

0.0    16004
1.0     3965
Name: TARGET_BUY, dtype: int64

Зададим списки признаков

In [31]:
num_pricebased = ['open', 'high', 'low', 'close', 'EMA_4_ema4o', 'EMA_5_ema5c', 'spread', 'tick_volume', 'CCI_14_0.015'] # to scale significally
num_indicator = [ 'RSI_14', 'MOM_14', ] # rsi/mom 0-100  maybe log scale  
num_indicator_stochastic = ['STOCHRSIk_14_14_3_3', 'STOCHRSId_14_14_3_3']

features = num_pricebased + num_indicator + num_indicator_stochastic
target = TARGET 

Separate 2 datasets, by class

In [33]:
df = df_full[features] 
df_buy = df.drop(columns=[TARGET_NEGATIVE], errors="ignore")
df_sell = df.drop(columns=[TARGET_POSITIVE], errors="ignore")

df_buy[TARGET] = df_full[TARGET_POSITIVE]
df_sell[TARGET] = df_full[TARGET_NEGATIVE]
df_sell.loc[df_sell[TARGET]==-1] = 1

In [34]:
df_buy.head(3)

,open,high,low,close,EMA_4_ema4o,EMA_5_ema5c,spread,tick_volume,CCI_14_0.015,RSI_14,MOM_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3,TARGET
31,1172.26,1172.93,1170.69,1172.68,1171.331451,1171.992824,160,3400,-60.901468,34.016815,-12.50,100.0,72.430645,0.0
32,1172.67,1174.32,1171.51,1173.93,1171.866871,1172.638549,160,2317,-25.768602,37.290539,-7.20,100.0,88.931516,0.0
33,1173.93,1178.60,1173.67,1177.65,1172.692122,1174.309033,160,4443,63.547341,45.893977,-1.33,100.0,100.000000,0.0


In [35]:
df_buy[TARGET].value_counts()

0.0    16004
1.0     3965
Name: TARGET, dtype: int64

In [36]:
df_sell[TARGET].value_counts()

0.0    15871
1.0     4098
Name: TARGET, dtype: int64

Разделим данные на train/test и сохраним тестовую выборку на диск (здесь мы ее касаться уже не будем)

In [38]:
# model for binary buy/do nothing classes
df = df_buy

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[TARGET]), df[TARGET], test_size=0.33, random_state=42)
#save test
X_test.to_csv(f"{DATA_PATH}X_test{BUY_SFX}.csv", index=None)
y_test.to_csv(f"{DATA_PATH}y_test{BUY_SFX}.csv", index=None)
#save train
X_train.to_csv(f"{DATA_PATH}X_train{BUY_SFX}.csv", index=None)
y_train.to_csv(f"{DATA_PATH}y_train{BUY_SFX}.csv", index=None)

In [39]:
X_train.describe()

,open,high,low,close,EMA_4_ema4o,EMA_5_ema5c,spread,tick_volume,CCI_14_0.015,RSI_14,MOM_14,STOCHRSIk_14_14_3_3,STOCHRSId_14_14_3_3
count,13379.000000,13379.000000,13379.000000,13379.000000,13379.000000,13379.000000,13379.000000,13379.000000,13379.000000,13379.000000,13379.000000,1.337900e+04,1.337900e+04
mean,1607.016260,1609.073221,1604.887482,1607.064999,1606.982425,1607.003562,151.644368,4517.627625,6.473103,51.391503,0.360410,5.057189e+01,5.052911e+01
std,242.631394,243.211263,241.976533,242.632724,242.632229,242.629579,19.569534,3125.522966,111.452364,12.552187,12.000844,3.225768e+01,3.102827e+01
min,1172.670000,1174.320000,1171.510000,1173.650000,1171.866871,1172.638549,0.000000,3.000000,-435.143714,10.276671,-101.440000,-2.107943e-13,-1.892807e-13
25%,1381.035000,1383.495000,1379.330000,1381.535000,1380.734758,1380.329949,140.000000,2140.000000,-76.230679,43.085182,-4.780000,2.055462e+01,2.169629e+01
50%,1698.990000,1701.650000,1696.400000,1698.950000,1698.954403,1699.100574,160.000000,3724.000000,9.698083,51.191261,0.560000,5.035184e+01,5.024155e+01
75%,1806.605000,1808.385000,1804.425000,1806.515000,1806.381045,1806.308272,160.000000,6054.500000,90.534497,59.994441,5.945000,8.073740e+01,7.972360e+01
max,2070.320000,2074.920000,2068.290000,2070.230000,2067.448537,2067.166346,1397.000000,20898.000000,409.487540,90.015744,79.764000,1.000000e+02,1.000000e+02


In [40]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13379, 13), (6590, 13), (13379,), (6590,))

In [41]:
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y= y_train)
class_weights

array([0.62355518, 2.5233874 ])

Соберем кусок, ответственный за feature engineering

In [42]:
# # Define categorical pipeline
# cat_pipe = Pipeline([
#     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
#     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
# ])

# Define numerical pipeline
price_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())    
])

indicator_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())  
    # ('scaler', MinMaxScaler())    
      # ('scaler', StandardScaler())
])

stoch_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# Combine categorical and numerical pipelines
preprocessor = ColumnTransformer([
      # ('cat', cat_pipe, cat_features),
      ('num_price', price_pipe, num_pricebased),
      ('num_ind', indicator_pipe, num_indicator) ,
      ('num_stoch', stoch_pipe, num_indicator_stochastic)   
  ],
remainder="drop")

In [ ]:
# import xgboost as xgb

Добавим простейший классификатор

In [43]:
%%time

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    # ('classifier', xgb.XGBClassifier(class_weight=class_weights, random_state=42, use_label_encoder=False)),
     # ('classifier', LogisticRegression(class_weight=class_weights, max_iter = 50000, random_state=42))
    ('classifier', CatBoostClassifier(iterations=100, thread_count=8, random_state=42, silent=True, train_dir=training_dir, eval_metric="F1"))   
])

pipeline.fit(X_train, y_train)

CPU times: user 1.33 s, sys: 41.4 ms, total: 1.37 s
Wall time: 948 ms


Посмотрим, как выглядит наш pipeline

In [44]:
pipeline.steps

[('preprocessor', ColumnTransformer(transformers=[('num_price',
                                   Pipeline(steps=[('imputer',
                                                    SimpleImputer(strategy='median')),
                                                   ('scaler', RobustScaler())]),
                                   ['open', 'high', 'low', 'close', 'EMA_4_ema4o',
                                    'EMA_5_ema5c', 'spread', 'tick_volume',
                                    'CCI_14_0.015']),
                                  ('num_ind',
                                   Pipeline(steps=[('imputer',
                                                    SimpleImputer(strategy='median')),
                                                   ('scaler', RobustScaler())]),
                                   ['RSI_14', 'MOM_14']),
                                  ('num_stoch',
                                   Pipeline(steps=[('imputer',
                                              

In [51]:
dmodel = {
    # "name": "logreg_pipeline",  
    "name": "catboost_pipeline",  
    "model": pipeline,
    "ver": "0.1",
    "date": datetime_to_title()
}

Сохраним модель (пайплайн)

In [52]:
with open(f"{DATA_PATH}catboost_pipeline.dill", "wb") as f:
    dill.dump(dmodel, f)

In [ ]:
# with open(f"{DATA_PATH}logreg_pipeline.dill", "wb") as f:
#     dill.dump(save_dict, f)

In [ ]:
# with open("xgb_pipeline.dill", "wb") as f:
#     dill.dump(pipeline, f)

In [50]:
!pip freeze > f"{DATA_PATH}requirements.txt"


/bin/bash: f/gdrive/My Drive/Ml_In_Business/kursovaya/requirements.txt: No such file or directory
